In [ ]:
# Fettoukh Mohamed Amine - Omri Ahmed

# ChatBot that generates reponses using seq2seq model

In [1]:
import re
import random
data_path = "human_text.txt"
data_path2 = "robot_text.txt"
# Defining lines as a list of each line
with open(data_path, 'r', encoding='utf-8') as f:
  lines = f.read().split('\n')
with open(data_path2, 'r', encoding='utf-8') as f:
  lines2 = f.read().split('\n')
lines = [re.sub(r"\[\w+\]",'hi',line) for line in lines]
lines = [" ".join(re.findall(r"\w+",line)) for line in lines]
lines2 = [re.sub(r"\[\w+\]",'',line) for line in lines2]
lines2 = [" ".join(re.findall(r"\w+",line)) for line in lines2]
# Grouping lines by response pair
pairs = list(zip(lines,lines2))
#random.shuffle(pairs)

In [2]:
import numpy as np

input_docs = []
target_docs = []
input_tokens = set()
target_tokens = set()
for line in pairs[:400]:
  input_doc, target_doc = line[0], line[1]
  # Appending each input sentence to input_docs
  input_docs.append(input_doc)
  # Splitting words from punctuation  
  target_doc = " ".join(re.findall(r"[\w']+|[^\s\w]", target_doc))
  # Redefine target_doc below and append it to target_docs
  target_doc = '<START> ' + target_doc + ' <END>'
  target_docs.append(target_doc)
  
  # Now we split up each sentence into words and add each unique word to our vocabulary set
  for token in re.findall(r"[\w']+|[^\s\w]", input_doc):
    if token not in input_tokens:
      input_tokens.add(token)
  for token in target_doc.split():
    if token not in target_tokens:
      target_tokens.add(token)
input_tokens = sorted(list(input_tokens))
target_tokens = sorted(list(target_tokens))
num_encoder_tokens = len(input_tokens)
num_decoder_tokens = len(target_tokens)

input_features_dict = dict(
    [(token, i) for i, token in enumerate(input_tokens)])
target_features_dict = dict(
    [(token, i) for i, token in enumerate(target_tokens)])

reverse_input_features_dict = dict(
    (i, token) for token, i in input_features_dict.items())
reverse_target_features_dict = dict(
    (i, token) for token, i in target_features_dict.items())


max_encoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", input_doc)) for input_doc in input_docs])
max_decoder_seq_length = max([len(re.findall(r"[\w']+|[^\s\w]", target_doc)) for target_doc in target_docs])

encoder_input_data = np.zeros(
    (len(input_docs), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_docs), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

for line, (input_doc, target_doc) in enumerate(zip(input_docs, target_docs)):
    for timestep, token in enumerate(re.findall(r"[\w']+|[^\s\w]", input_doc)):
        #Assign 1. for the current line, timestep, & word in encoder_input_data
        encoder_input_data[line, timestep, input_features_dict[token]] = 1.
    
    for timestep, token in enumerate(target_doc.split()):
        decoder_input_data[line, timestep, target_features_dict[token]] = 1.
        if timestep > 0:
            decoder_target_data[line, timestep - 1, target_features_dict[token]] = 1.

In [3]:
print(pairs[:5])
print(input_docs[:5])

[('hi', 'hi there how are you'), ('oh thanks i m fine this is an evening in my timezone', 'here is afternoon'), ('how do you feel today tell me something about yourself', 'my name is rdany but you can call me dany the r means robot i hope we can be virtual friends'), ('how many virtual friends have you got', 'i have many but not enough to fully understand humans beings'), ('is that forbidden for you to tell the exact number', 'i ve talked with 143 users counting 7294 lines of text')]
['hi', 'oh thanks i m fine this is an evening in my timezone', 'how do you feel today tell me something about yourself', 'how many virtual friends have you got', 'is that forbidden for you to tell the exact number']


In [7]:
!pip install tensorflow


from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
#Dimensionality
dimensionality = 256
#The batch size and number of epochs
batch_size = 10
epochs = 600
#Encoder
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(dimensionality, return_state=True)
encoder_outputs, state_hidden, state_cell = encoder_lstm(encoder_inputs)
encoder_states = [state_hidden, state_cell]
#Decoder
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(dimensionality, return_sequences=True, return_state=True)
decoder_outputs, decoder_state_hidden, decoder_state_cell = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#Model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
#Compiling
training_model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], sample_weight_mode='temporal')
#Training
training_model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size = batch_size, epochs = epochs, validation_split = 0.2)
training_model.save('training_model.h5')

  Using cached tensorboard-2.7.0-py3-none-any.whl (5.8 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached importlib_metadata-4.8.2-py3-none-any.whl (17 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.0
    Uninstalling importlib-metadata-3.10.0:
      Successfully uninstalled importlib-metadata-3.10.0
Epoch 1/600
32/32 [==============================] - 10s 258ms/step - loss: 1.2528 - accuracy: 0.0227 - val_loss: 1.3581 - val_accuracy: 0.0200
Epoch 2/600
32/32 [==============================] - 7s 218ms/step - loss: 1.1848 - accuracy: 0.0237 - val_loss

Epoch 16/600
32/32 [==============================] - 7s 214ms/step - loss: 1.1614 - accuracy: 0.0244 - val_loss: 1.6018 - val_accuracy: 0.0198
Epoch 17/600
32/32 [==============================] - 7s 216ms/step - loss: 1.1583 - accuracy: 0.0253 - val_loss: 1.6122 - val_accuracy: 0.0203
Epoch 18/600
32/32 [==============================] - 7s 224ms/step - loss: 1.1595 - accuracy: 0.0243 - val_loss: 1.6219 - val_accuracy: 0.0207
Epoch 19/600
32/32 [==============================] - 7s 209ms/step - loss: 1.1559 - accuracy: 0.0251 - val_loss: 1.6324 - val_accuracy: 0.0200
Epoch 20/600
32/32 [==============================] - 7s 217ms/step - loss: 1.1543 - accuracy: 0.0254 - val_loss: 1.6385 - val_accuracy: 0.0205
Epoch 21/600
32/32 [==============================] - 7s 217ms/step - loss: 1.1533 - accuracy: 0.0253 - val_loss: 1.6475 - val_accuracy: 0.0203
Epoch 22/600
32/32 [==============================] - 7s 217ms/step - loss: 1.1537 - accuracy: 0.0257 - val_loss: 1.6528 - val_accuracy:

Epoch 73/600
32/32 [==============================] - 6s 182ms/step - loss: 1.1212 - accuracy: 0.0266 - val_loss: 1.6837 - val_accuracy: 0.0200
Epoch 74/600
32/32 [==============================] - 6s 181ms/step - loss: 1.1170 - accuracy: 0.0276 - val_loss: 1.6829 - val_accuracy: 0.0203
Epoch 75/600
32/32 [==============================] - 6s 186ms/step - loss: 1.1164 - accuracy: 0.0274 - val_loss: 1.6784 - val_accuracy: 0.0203
Epoch 76/600
32/32 [==============================] - 6s 188ms/step - loss: 1.1178 - accuracy: 0.0273 - val_loss: 1.6835 - val_accuracy: 0.0198
Epoch 77/600
32/32 [==============================] - 6s 185ms/step - loss: 1.1178 - accuracy: 0.0272 - val_loss: 1.6802 - val_accuracy: 0.0200
Epoch 78/600
32/32 [==============================] - 6s 185ms/step - loss: 1.1106 - accuracy: 0.0282 - val_loss: 1.6844 - val_accuracy: 0.0192
Epoch 79/600
32/32 [==============================] - 6s 182ms/step - loss: 1.1113 - accuracy: 0.0278 - val_loss: 1.6837 - val_accuracy:

Epoch 130/600
32/32 [==============================] - 6s 183ms/step - loss: 1.0806 - accuracy: 0.0316 - val_loss: 1.7040 - val_accuracy: 0.0203
Epoch 131/600
32/32 [==============================] - 6s 186ms/step - loss: 1.0819 - accuracy: 0.0311 - val_loss: 1.7007 - val_accuracy: 0.0192
Epoch 132/600
32/32 [==============================] - 6s 185ms/step - loss: 1.0876 - accuracy: 0.0304 - val_loss: 1.6983 - val_accuracy: 0.0200
Epoch 133/600
32/32 [==============================] - 6s 182ms/step - loss: 1.0830 - accuracy: 0.0310 - val_loss: 1.7099 - val_accuracy: 0.0210
Epoch 134/600
32/32 [==============================] - 6s 184ms/step - loss: 1.0796 - accuracy: 0.0307 - val_loss: 1.7125 - val_accuracy: 0.0200
Epoch 135/600
32/32 [==============================] - 6s 180ms/step - loss: 1.0754 - accuracy: 0.0318 - val_loss: 1.7166 - val_accuracy: 0.0203
Epoch 136/600
32/32 [==============================] - 6s 180ms/step - loss: 1.0749 - accuracy: 0.0312 - val_loss: 1.7111 - val_ac

32/32 [==============================] - 6s 180ms/step - loss: 0.9850 - accuracy: 0.0463 - val_loss: 1.7492 - val_accuracy: 0.0210
Epoch 243/600
32/32 [==============================] - 6s 177ms/step - loss: 0.9812 - accuracy: 0.0463 - val_loss: 1.7479 - val_accuracy: 0.0203
Epoch 244/600
32/32 [==============================] - 6s 178ms/step - loss: 0.9804 - accuracy: 0.0463 - val_loss: 1.7361 - val_accuracy: 0.0210
Epoch 245/600
32/32 [==============================] - 6s 180ms/step - loss: 0.9936 - accuracy: 0.0454 - val_loss: 1.7580 - val_accuracy: 0.0207
Epoch 246/600
32/32 [==============================] - 6s 193ms/step - loss: 0.9843 - accuracy: 0.0457 - val_loss: 1.7419 - val_accuracy: 0.0200
Epoch 247/600
32/32 [==============================] - 7s 208ms/step - loss: 0.9926 - accuracy: 0.0441 - val_loss: 1.7509 - val_accuracy: 0.0207
Epoch 248/600
32/32 [==============================] - 6s 198ms/step - loss: 0.9768 - accuracy: 0.0472 - val_loss: 1.7378 - val_accuracy: 0.0215

32/32 [==============================] - 6s 177ms/step - loss: 0.8410 - accuracy: 0.0697 - val_loss: 1.7823 - val_accuracy: 0.0215
Epoch 355/600
32/32 [==============================] - 6s 183ms/step - loss: 0.8404 - accuracy: 0.0701 - val_loss: 1.7739 - val_accuracy: 0.0220
Epoch 356/600
32/32 [==============================] - 6s 179ms/step - loss: 0.8365 - accuracy: 0.0704 - val_loss: 1.7764 - val_accuracy: 0.0215
Epoch 357/600
32/32 [==============================] - 6s 178ms/step - loss: 0.8372 - accuracy: 0.0706 - val_loss: 1.7755 - val_accuracy: 0.0215
Epoch 358/600
32/32 [==============================] - 6s 180ms/step - loss: 0.8540 - accuracy: 0.0677 - val_loss: 1.7610 - val_accuracy: 0.0220
Epoch 359/600
32/32 [==============================] - 6s 178ms/step - loss: 0.8326 - accuracy: 0.0719 - val_loss: 1.7734 - val_accuracy: 0.0223
Epoch 360/600
32/32 [==============================] - 6s 178ms/step - loss: 0.8286 - accuracy: 0.0716 - val_loss: 1.7803 - val_accuracy: 0.0213

32/32 [==============================] - 6s 180ms/step - loss: 0.6938 - accuracy: 0.0979 - val_loss: 1.8197 - val_accuracy: 0.0207
Epoch 467/600
32/32 [==============================] - 6s 196ms/step - loss: 0.6921 - accuracy: 0.0974 - val_loss: 1.7982 - val_accuracy: 0.0213
Epoch 468/600
32/32 [==============================] - 6s 186ms/step - loss: 0.6907 - accuracy: 0.0969 - val_loss: 1.8127 - val_accuracy: 0.0198
Epoch 469/600
32/32 [==============================] - 6s 180ms/step - loss: 0.6946 - accuracy: 0.0974 - val_loss: 1.8047 - val_accuracy: 0.0200
Epoch 470/600
32/32 [==============================] - 6s 192ms/step - loss: 0.6814 - accuracy: 0.1001 - val_loss: 1.8107 - val_accuracy: 0.0195
Epoch 471/600
32/32 [==============================] - 6s 194ms/step - loss: 0.6844 - accuracy: 0.0991 - val_loss: 1.8097 - val_accuracy: 0.0200
Epoch 472/600
32/32 [==============================] - 7s 214ms/step - loss: 0.6824 - accuracy: 0.0999 - val_loss: 1.8052 - val_accuracy: 0.0215

32/32 [==============================] - 6s 182ms/step - loss: 0.5929 - accuracy: 0.1152 - val_loss: 1.8315 - val_accuracy: 0.0215
Epoch 579/600
32/32 [==============================] - 6s 178ms/step - loss: 0.5949 - accuracy: 0.1155 - val_loss: 1.8319 - val_accuracy: 0.0215
Epoch 580/600
32/32 [==============================] - 6s 179ms/step - loss: 0.5895 - accuracy: 0.1159 - val_loss: 1.8507 - val_accuracy: 0.0213
Epoch 581/600
32/32 [==============================] - 6s 178ms/step - loss: 0.5892 - accuracy: 0.1168 - val_loss: 1.8408 - val_accuracy: 0.0217
Epoch 582/600
32/32 [==============================] - 6s 177ms/step - loss: 0.5846 - accuracy: 0.1172 - val_loss: 1.8202 - val_accuracy: 0.0210
Epoch 583/600
32/32 [==============================] - 6s 178ms/step - loss: 0.6022 - accuracy: 0.1140 - val_loss: 1.8308 - val_accuracy: 0.0220
Epoch 584/600
32/32 [==============================] - 6s 180ms/step - loss: 0.5976 - accuracy: 0.1151 - val_loss: 1.8455 - val_accuracy: 0.0213

In [8]:
from keras.models import load_model
training_model = load_model('training_model.h5')
encoder_inputs = training_model.input[0]
encoder_outputs, state_h_enc, state_c_enc = training_model.layers[2].output
encoder_states = [state_h_enc, state_c_enc]
encoder_model = Model(encoder_inputs, encoder_states)

latent_dim = 256
decoder_state_input_hidden = Input(shape=(latent_dim,))
decoder_state_input_cell = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_hidden, decoder_state_input_cell]
decoder_outputs, state_hidden, state_cell = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_hidden, state_cell]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

def decode_response(test_input):
    #Getting the output states to pass into the decoder
    states_value = encoder_model.predict(test_input)
    #Generating empty target sequence of length 1
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    #Setting the first token of target sequence with the start token
    target_seq[0, 0, target_features_dict['<START>']] = 1.
    
    #A variable to store our response word by word
    decoded_sentence = ''
    
    stop_condition = False
    while not stop_condition:
          #Predicting output tokens with probabilities and states
          output_tokens, hidden_state, cell_state = decoder_model.predict([target_seq] + states_value)
    #Choosing the one with highest probability
          sampled_token_index = np.argmax(output_tokens[0, -1, :])
          sampled_token = reverse_target_features_dict[sampled_token_index]
          decoded_sentence += " " + sampled_token
    #Stop if hit max length or found the stop token
          if (sampled_token == '<END>' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
    #Update the target sequence
          target_seq = np.zeros((1, 1, num_decoder_tokens))
          target_seq[0, 0, sampled_token_index] = 1.
          #Update states
          states_value = [hidden_state, cell_state]
    return decoded_sentence

In [ ]:
class ChatBot:
  negative_responses = ("no", "nope", "nah", "naw", "not a chance", "sorry")
  exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later", "stop")
    
#Method to start the conversation
  def start_chat(self):
    user_response = input("Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?\n")
    
    if user_response in self.negative_responses:
      print("Ok, have a great day!")
      return
    self.chat(user_response)
    
#Method to handle the conversation
  def chat(self, reply):
    while not self.make_exit(reply):
      reply = input(self.generate_response(reply)+"\n")
    
  #Method to convert user input into a matrix
  def string_to_matrix(self, user_input):
    tokens = re.findall(r"[\w']+|[^\s\w]", user_input)
    user_input_matrix = np.zeros(
      (1, max_encoder_seq_length, num_encoder_tokens),
      dtype='float32')
    for timestep, token in enumerate(tokens):
      if token in input_features_dict:
        user_input_matrix[0, timestep, input_features_dict[token]] = 1.
    return user_input_matrix
  
  #Method that will create a response using seq2seq model we built
  def generate_response(self, user_input):
    input_matrix = self.string_to_matrix(user_input)
    chatbot_response = decode_response(input_matrix)
    #Remove <START> and <END> tokens from chatbot_response
    chatbot_response = chatbot_response.replace("<START>",'')
    chatbot_response = chatbot_response.replace("<END>",'')
    return chatbot_response

#Method to check for exit commands
  def make_exit(self, reply):
    for exit_command in self.exit_commands:
      if exit_command in reply:
        print("Ok, have a great day!")
        return True
    return False
  
chatbot = ChatBot()
chatbot.start_chat()

Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?
hi there


In [0]:
chatbot.start_chat()

Hi, I'm a chatbot trained on random dialogs. Would you like to chat with me?
hi there
 hi there how are you today 
i m fine
 everything is fine on this side 
nice
 how is work going today 
all is good
 i m trying to learn like with the ukulele 
okay bye
Ok, have a great day!
